In [9]:
from shapely.geometry import Point

def shapely_find(line):
    nums = list(map(float, line.split()))
    c1 = Point(nums[0], nums[1]).buffer(nums[2], resolution=1000)
    c2 = Point(nums[3], nums[4]).buffer(nums[5], resolution=1000)
    c3 = Point(nums[6], nums[7]).buffer(nums[8], resolution=1000)
    intersection = c1.intersection(c2).intersection(c3)
    return intersection.area

file_path = 'data/585_intersection_of_circles/data.txt'
answer_file_path = 'data/585_intersection_of_circles/answer.txt'

with open(file_path, 'r') as f_in, open(answer_file_path, 'w') as f_out:
    for line in f_in:
        area = shapely_find(line)
        f_out.write(f"{area:.7f}\n")

In [15]:
import math

def get_intersections(x1, y1, r1, x2, y2, r2):
    d2 = (x1 - x2)**2 + (y1 - y2)**2
    d = math.sqrt(d2)
    if d > r1 + r2 or d < abs(r1 - r2) or d == 0:
        return []
    a = (r1**2 - r2**2 + d2) / (2 * d)
    h = math.sqrt(max(0, r1**2 - a**2))
    x0 = x1 + a * (x2 - x1) / d
    y0 = y1 + a * (y2 - y1) / d
    rx = -(y2 - y1) * (h / d)
    ry = (x2 - x1) * (h / d)
    return [(x0 + rx, y0 + ry), (x0 - rx, y0 - ry)]

def get_area_contribution(x, y, r, a1, a2, cx, cy):
    return 0.5 * r * (cx * (math.sin(a2) - math.sin(a1)) - cy * (math.cos(a2) - math.cos(a1))) + 0.5 * r**2 * (a2 - a1)

def solve_analytic(line):
    parts = list(map(float, line.split()))
    c = [(parts[0], parts[1], parts[2]), (parts[3], parts[4], parts[5]), (parts[6], parts[7], parts[8])]
    
    total_area = 0
    for i in range(3):
        circles_to_check = [(c[(i+1)%3]), (c[(i+2)%3])]
        angles = []
        for j in range(1, 3):
            other = c[(i+j)%3]
            pts = get_intersections(c[i][0], c[i][1], c[i][2], other[0], other[1], other[2])
            for p in pts:
                angles.append(math.atan2(p[1] - c[i][1], p[0] - c[i][0]))
        
        angles = sorted(list(set(angles)))
        for j in range(len(angles)):
            a1, a2 = angles[j], angles[(j+1) % len(angles)]
            if a2 < a1: a2 += 2 * math.pi
            
            mid_angle = (a1 + a2) / 2
            test_p = (c[i][0] + c[i][2] * math.cos(mid_angle), c[i][1] + c[i][2] * math.sin(mid_angle))

            is_inside = True
            for k in range(1, 3):
                other = c[(i+k)%3]
                if (test_p[0]-other[0])**2 + (test_p[1]-other[1])**2 > other[2]**2 + 1e-9:
                    is_inside = False
                    break
            
            if is_inside:
                total_area += get_area_contribution(test_p[0], test_p[1], c[i][2], a1, a2, c[i][0], c[i][1])
                
    if total_area == 0:
        c_sorted = sorted(c, key=lambda x: x[2])
        small = c_sorted[0]
        if all((small[0]-o[0])**2 + (small[1]-o[1])**2 <= (o[2]-small[2])**2 + 1e-9 for o in c if o != small):
            return math.pi * small[2]**2  
    return abs(total_area)

file_path = 'data/585_intersection_of_circles/data.txt'
answer_file_path = 'data/585_intersection_of_circles/answer_math.txt'

with open(file_path, 'r') as f_in, open(answer_file_path, 'w') as f_out:
    for line in f_in:
        area = solve_analytic(line)
        f_out.write(f"{area:.7f}\n")